In [921]:
from __future__ import division, print_function
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn; seaborn.set()  # plot styling

In [922]:
def geometric_center(coordinates):
    """
    coordinates: np.array
                 coordinates of one atom per line [x,y,z] 
    return: np.array
            coordinates of the geometric center
    """
    return np.sum(coordinates, axis=0) / len(coordinates)

def radius(center, coordinates):
    """
    center: np.array
            coordinates of the center of the sphere [x,y,z]
    coordinates: np.array
                 coordinates of one atom per line [x,y,z] (the atoms are on the surface of the sphere)
    return: np.array
            radius
    """
    distances = (coordinates - center)**2
    radius = (np.sum(np.sqrt(distances.sum(axis=1)))) / len(distances)
    return radius

def distances(center, coordinates):
    """
    center: np.array
            coordinates of the center of the sphere [x,y,z]
    coordinates: np.array
                 coordinates of one atom per line [x,y,z]
    return: np.array
            distances between the center and the atoms (one distance per atom)
    """
    distances = (coordinates - center)**2
    # if we calculate the distance from the center for only one atom:
    if len(distances.shape) == 1:
        distances = np.sqrt(distances.sum())
    else:
        distances = np.sqrt(distances.sum(axis=1))
    return distances

def cartesian_to_spherical_coordinates(coordinates):
    """
    coordinates: np.array
                 coordinates of one atom per line [x,y,z]
    return: np.array
            spherical coordinates of one atom per line [r,theta,phi]
    """
    if len(coordinates.shape) == 1:
        x = coordinates[0]
        y = coordinates[1]
        z = coordinates[2]
        r = np.sqrt(x**2+y**2+z**2)
        theta = np.arccos(z/r)
        phi = np.arctan(y/x)
        return np.array([r, theta, phi])
    else:
        x = coordinates[:, 0]
        y = coordinates[:, 1]
        z = coordinates[:, 2]
        r = np.sqrt(x**2+y**2+z**2)
        theta = np.arccos(z/r)
        phi = np.arctan(y/x)
        r = np.expand_dims(r, axis=1)
        theta = np.expand_dims(theta, axis=1)
        phi = np.expand_dims(phi, axis=1)
        return np.hstack((r, theta, phi))

def spherical_to_cartesian_coordinates(spherical_coordinates):
    """
    spherial_coordinates: np.array
                          spherical coordinates of one atom per line [r,theta,phi]
    return: np.array
            cartesian coordinates of one atom per line [x,y,z]
    """
    if len(spherical_coordinates.shape) == 1:
        r = spherical_coordinates[0]
        theta = spherical_coordinates[1]
        phi = spherical_coordinates[2]
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        return np.array([x, y, z])
    else:
        r = spherical_coordinates[:, 0]
        theta = spherical_coordinates[:, 1]
        phi = spherical_coordinates[:, 2]
        x = r * np.sin(theta) * np.cos(phi)
        y = r * np.sin(theta) * np.sin(phi)
        z = r * np.cos(theta)
        x = np.expand_dims(x, axis=1)
        y = np.expand_dims(y, axis=1)
        z = np.expand_dims(z, axis=1)
        return np.hstack((x, y, z))

def normalize(v, tolerance=0.00001):
    """
    v: list or np.array
        vector
    return: np.array
            normalized vector
    """
    magnitude_square = np.sum(n * n for n in v)
    if abs(magnitude_square - 1.0) > tolerance:
        magnitude = np.sqrt(magnitude_square)
        v = [n / magnitude for n in v]
    return np.array(v)

def axisangle_to_quaternion(axis, angle):
    """
    axis: list or np.array (3,)
          axis (= vector) of rotation
    angle: float in radian
           angle of rotation
    return: np.array (4,)
            quaternion
    """
    axis = normalize(axis)
    x, y, z = axis
    angle /= 2.
    w = np.cos(angle)
    x = x * np.sin(angle)
    y = y * np.sin(angle)
    z = z * np.sin(angle)
    return np.array([w, x, y, z])

class Quaternion(object):
    def __init__(self, quaternion):
        """
        quaternion: list or np.array (4,)
        return: np.array (4,) [w, x, y, z]
        """
        #self.w = quaternion[0]
        #self.x = quaternion[1]
        #self.y = quaternion[2]
        #self.z = quaternion[3]
        self.q = np.array([quaternion[0], quaternion[1], quaternion[2], quaternion[3]])
    
    def __str__(self):
        """
        How it will appear with the print function
        """
        return('quaternion(%f, %f, %f, %f)' % (self.q[0], self.q[1], self.q[2], self.q[3]))
    
    def __repr__(self):
        """
        Override __repr__: the quaternion will appear as a numpy array when you type the instance in the notebook
        """
        return repr(self.q)
    
    def __getitem__(self, index):
        """
        Allows its instances to use the [] (indexer) operators.
        """
        return self.q[index]

    def __mul__(self, quaternion2):
        """
        Multiplication of quaternions
        """
        w = self.q[0] * quaternion2.q[0] - self.q[1] * quaternion2.q[1] - self.q[2] * quaternion2.q[2] - self.q[3] * quaternion2.q[3]
        x = self.q[0] * quaternion2.q[1] + self.q[1] * quaternion2.q[0] + self.q[2] * quaternion2.q[3] - self.q[3] * quaternion2.q[2]
        y = self.q[0] * quaternion2.q[2] + self.q[2] * quaternion2.q[0] + self.q[3] * quaternion2.q[1] - self.q[1] * quaternion2.q[3]
        z = self.q[0] * quaternion2.q[3] + self.q[3] * quaternion2.q[0] + self.q[1] * quaternion2.q[2] - self.q[2] * quaternion2.q[1]
        return Quaternion([w, x, y, z])
    
    def conjugate(self):
        return(Quaternion(np.array([self.q[0], -self.q[1], -self.q[2], -self.q[3]])))
    
    def rotate_vector(self, vector):
        """
        Rotation of a vector == multiplication of a quaternion by a vector:
        q * vector * q.conjugate()
        vector: list or np.array (3,)
        """
        # if vector is a list, convert to a numpy array
        if not('numpy' in str(type(vector))):
            vector = np.array(vector)
        #Converting vector in a quaternion with w = 0:
        vector = np.append([0.0,], vector) 
        vector = np.array(vector)
        vector = Quaternion(vector)
        res = (self*vector)*self.conjugate()
        return(res[1:])
            
    def rotation_matrix(self):
        """
        return: np.array
                quaternion-derived rotation matrix (3, 3)
        """
        matrix = np.zeros((3, 3))
        # calculate the rotation matrix based on the quaternion value
        matrix[0, 0] = self.q[0]**2 + self.q[1]**2 - self.q[2]**2 - self.q[3]**2
        matrix[0, 1] = 2 * (self.q[1]*self.q[2] + self.q[0]*self.q[3])
        matrix[0, 2] = 2 * (self.q[1]*self.q[3] - self.q[0]*self.q[2])
        matrix[1, 0] = 2 * (self.q[1]*self.q[2] - self.q[0]*self.q[2])
        matrix[1, 1] = self.q[0]**2 - self.q[1]**2 + self.q[2]**2 - self.q[3]**2
        matrix[1, 2] = 2 * (self.q[2]*self.q[3] + self.q[0]*self.q[1])
        matrix[2, 0] = 2 * (self.q[1]*self.q[3] + self.q[0]*self.q[2])
        matrix[2, 1] = 2 * (self.q[2]*self.q[3] - self.q[0]*self.q[1])
        matrix[2, 2] = self.q[0]**2 - self.q[1]**2 - self.q[2]**2 + self.q[3]**2
    
        return matrix

In [923]:
# function to automate the rotation
def rotate(axis_of_rotation, angle_of_rotation, coordinates):
    """
    axis_of_rotation: list
    angle_of_rotation: radian
    coordinates: np.array
                 coordinates of the molecule to rotate
    """
    q = axisangle_to_quaternion(axis_of_rotation, angle_of_rotation) # the quaternion is also converted to a unit quaternion
    # define as a quaternion
    quaternion = Quaternion(q)
    new_coordinates = np.zeros((len(coordinates), 3))
    if coordinates.ndim == 1:
        new_coordinates = quaternion.rotate_vector(coordinates)
    else:
        for i, vector in enumerate(coordinates):
            new_coordinates[i] = quaternion.rotate_vector(vector)
    # Note: can also use the quaternion-derived rotation matrix to rotate the vector
    #if coordinates.ndim == 1:
    #    new_coordinates = quaternion.rotation_matrix().dot(coordinates)
    #else:
    #    for i, vector in enumerate(coordinates):
    #        new_coordinates[i] = quaternion.rotation_matrix().dot(vector)
    return(new_coordinates)

# Testing

In [924]:
def test_rotation():
    # Test 1:
    axis_of_rotation = [0., 0., 1.]
    angle_of_rotation = np.pi
    vector_to_rotate = np.array([1., 0. ,0.])
    solution = np.array([-1., 0., 0.])
    np.testing.assert_array_almost_equal(rotate(axis_of_rotation, angle_of_rotation, vector_to_rotate), solution, decimal = 15)  
    # Test 2:
    vector_to_rotate = np.array([1., 1. ,0.])
    solution = np.array([-1., -1., 0.])
    np.testing.assert_array_almost_equal(rotate(axis_of_rotation, angle_of_rotation, vector_to_rotate), solution, decimal = 15) 

In [925]:
if test_rotation() == None:
    print('Tests OK!')
else:
    test_rotation()

Tests OK!


# Application on Caveolin

In [926]:
%cd /home/chevrot/Simulations/Caveolin/Lipid_droplets/CO-30_DO-20_Cav_surface/1_preparation/CG_centroid_1
file_name = 'CG1_center.gro'

/home/chevrot/Simulations/Caveolin/Lipid_droplets/CO-30_DO-20_Cav_surface/1_preparation/CG_centroid_1


In [927]:
with open(file_name, 'r') as f:
    data = f.readlines()

In [928]:
coordinates = []
for line in data:
    coordinates.append(line.split()[-3:]) 
# 2 first and last lines are empty array
coordinates = coordinates[2:-1]
len(coordinates)

138

In [929]:
coordinates = np.array(coordinates)
# transform string to float
coordinates = coordinates.astype(np.float)

In [930]:
coordinates.shape

(138, 3)

##### Determining the position of the sphere and the radius of the sphere

I want to put 18 caveolin molecules on the surface of the lipid droplet

In [931]:
coord = np.zeros((138, 3, 18))

In [932]:
file_name = '../equi.gro'

In [933]:
with open(file_name, 'r') as f:
    data_ld = f.readlines()

In [934]:
sphere_coordinates = []
for line in data_ld[2:-1]: # 2 first and last lines are not atoms and coordinates
    if re.match(r'PO4[1-9]+', line.split()[1]):
        sphere_coordinates.append(line.split()[-6:-3]) 
sphere_coordinates = np.array(sphere_coordinates)
sphere_coordinates = sphere_coordinates.astype(np.float)
sphere_coordinates.shape

(2143, 3)

In [935]:
center_of_sphere = geometric_center(sphere_coordinates)
center_of_sphere

array([ 14.58679701,  16.50979701,  15.69261549])

In [936]:
sphere_radius = radius(center_of_sphere, sphere_coordinates)
sphere_radius

11.844919353857689

##### Translating Caveolin to origin (needed before rotation) 

In [937]:
geometric_center(coordinates)

array([ 14.79631884,  14.79636957,  14.79633333])

In [938]:
coordinates = coordinates - geometric_center(coordinates)

In [939]:
coordinates.shape

(138, 3)

##### Rotation to put the "main axis" of caveolin in the z-direction

In [940]:
new_coordinates = rotate([0,1,0], -np.pi/4., coordinates)

Translate the coordinates on the surface of the lipid droplet

In [941]:
radius = sphere_radius

In [942]:
cpt = 0 # counter - from 0 to 17 <==> number of caveolins on the sphere
translate = np.array([0., 0., radius])
translate += center_of_sphere
coord[:,:,cpt] = new_coordinates + translate
cpt+=1

##### Rotation around x-axis and translation on a sphere

In [943]:
axis_of_rotation = [1., 0., 0.]
angle_of_rotation = np.pi

# Define the translation vector:
main_axis_of_the_molecule = np.array([0., 0., 1.])
translate = radius * rotate(axis_of_rotation, angle_of_rotation, main_axis_of_the_molecule)
translate += center_of_sphere

# Rotating and translating on the surface of the sphere
coord[:,:,cpt] = rotate(axis_of_rotation, angle_of_rotation, new_coordinates)  + translate
cpt+=1

##### Rotation around x-axis and translation on a sphere

In [944]:
axis_of_rotation = [1., 0. , 0]
main_axis_of_the_molecule = np.array([0., 0., 1.])

for angle_of_rotation in (np.pi/4., np.pi/2, 3*np.pi/4., -np.pi/4., -np.pi/2, -3*np.pi/4.):
    # Define the translation vector:
    translate = radius * rotate(axis_of_rotation, angle_of_rotation, main_axis_of_the_molecule)
    translate += center_of_sphere
    # Rotating and translating on the surface of the sphere
    coord[:,:,cpt] = rotate(axis_of_rotation, angle_of_rotation, new_coordinates)  + translate
    cpt+=1

##### Rotation around y-axis and translation on a sphere

In [945]:
axis_of_rotation = [0., 1. , 0]
main_axis_of_the_molecule = np.array([0., 0., 1.])

for angle_of_rotation in (np.pi/4., np.pi/2., 3*np.pi/4., -np.pi/4., -np.pi/2., -3*np.pi/4.):
    # Define the translation vector:
    translate = radius * rotate(axis_of_rotation, angle_of_rotation, main_axis_of_the_molecule)
    translate += center_of_sphere
    # Rotating and translating on the surface of the sphere
    coord[:,:,cpt] = rotate(axis_of_rotation, angle_of_rotation, new_coordinates)  + translate
    cpt+=1

Now, we have 8 + 6 caveolins distributed around 2 perpendicular circles.
I want 2+2 more caveolins on latitudes 45$^{\circ}$, -45$^{\circ}$.

##### Rotation and translation on a sphere

In [946]:
axis_of_rotation = normalize(spherical_to_cartesian_coordinates(np.array([1., np.pi/4, 0.])))
for angle_of_rotation in (3*np.pi/4., -3*np.pi/4.):
    # Define the translation vector:
    translate = radius * rotate(axis_of_rotation, angle_of_rotation, main_axis_of_the_molecule)
    translate += center_of_sphere
    # Rotating and translating on the surface of the sphere
    coord[:,:,cpt] = rotate(axis_of_rotation, angle_of_rotation, new_coordinates)  + translate
    cpt+=1

In [947]:
axis_of_rotation = normalize(spherical_to_cartesian_coordinates(np.array([1., -np.pi/4, 0.])))
for angle_of_rotation in (3*np.pi/4., -3*np.pi/4.):
    # Define the translation vector:
    translate = radius * rotate(axis_of_rotation, angle_of_rotation, main_axis_of_the_molecule)
    translate += center_of_sphere
    # Rotating and translating on the surface of the sphere
    coord[:,:,cpt] = rotate(axis_of_rotation, angle_of_rotation, new_coordinates)  + translate
    cpt+=1


# Writing a new gro file

In [950]:
# Number of atoms (18 caveolins)
natoms = len(coordinates_1)*18
natoms

2484

In [951]:
file_name = 'out_18cav.gro'
# Open a new file
with open(file_name, 'w') as outfile:
    # the first line is let inchanged
    for line in data[0]:
        outfile.writelines(line)
    # write the number of atoms
    outfile.write(str(natoms)+'\n')
    # flatten the coordinates
    for i in range(coord.shape[-1]):
        for line, xyz in zip(data[2:-1], coord[:,:,i]):
            outfile.writelines(line[:22]) # write the name and number of the atoms
            outfile.writelines('{0:6.3f} {1:7.3f} {2:7.3f}\n'.format(xyz[0], xyz[1], xyz[2]))
    for line in data[-1]:
        outfile.writelines(line)

In [919]:
file_name = 'origin.gro'
# Open a new file
with open(file_name, 'w') as outfile:
    # the first lines (==proteins) are let inchanged
    for line in data[0:2]:
        outfile.writelines(line)
    for line, coord in zip(data[2:-1], new_coordinates):
        outfile.writelines(line[:22]) # write the name and number of the atoms
        outfile.writelines('{0:.3f} {1:.3f} {2:.3f}\n'.format(coord[0], coord[1], coord[2]))
    for line in data[-1]:
        outfile.writelines(line)